In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from datetime import datetime, time
import pandas_datareader.data as pdr
import yfinance as yf 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [3]:
def stock_data(ticker, start):
    try:
        prices = yf.download(ticker, start, auto_adjust=True)['Close']
    except Exception as e:
        print(f"Error: {e}")
    return prices
    

In [108]:
def dual_momemtum(prices, stocks, market, mom_lookback=6, ma_window = 10, top_n=5):
    """Calculating 6 month dual momentum with monthly rebalancing"""


     #Monthly rebalancing to trade monthly
    prices_rb = prices.resample("ME").last()
   

    #Absolute Momemtum for the market
    abs_mom = prices_rb[market].pct_change(mom_lookback)
    trend = prices_rb[market] > prices_rb[market].rolling(ma_window).mean()
    market_signal = (abs_mom > 0) & trend

    #print(market_signal.mean()) #debug

    #Relative momentum for stocks
    stocks_return = prices_rb[stocks].pct_change(mom_lookback)

    #Ranking stocks
    ranks = stocks_return.rank(axis = 1, ascending= False, method='first')
    top_stocks = ranks <= top_n

   
    #monthly weights
    weights_rb = pd.DataFrame(0.0, index= prices_rb.index, columns=prices.columns)
    "this step is initializing weight matrix. rows are rebalance dates . Cash is represented as all zeros"

    for date in prices_rb.index:
        if market_signal.loc[date]:
            selected = top_stocks.loc[date]
            if selected.sum() > 0:
                weights_rb.loc[date, stocks] = (selected / selected.sum()) #assigning equal weight to selected stocks

    #Daily tradable weights
    weights = ( weights_rb.reindex(prices.index).ffill().shift(1).fillna(0))

    """  reindex -> expands weights from monthly to daily and add Nads to non-rebalace days.
    ffill() - forward-fills last rebelance weights and portfolio is held constant between rebalance dates
    shift(1) - signals use today's close price (prevent lookahead bias)
    fillna(0) - before first signal. everything is in cash which is presented by 0.  """

    #Strategy returns
    returns = prices.pct_change()
    strategy_returns =( weights * returns).sum(axis = 1)
    equity_curve = (1+ strategy_returns).cumprod()

    #print(weights.sum(axis=1).mean()) #debug
    #print(strategy_returns.std()) #debug

    #Plot Ranks
    fig1 = go.Figure()

    for col in ranks.columns:
        fig1.add_trace(go.Scatter(x=ranks.index,
                                 y=ranks[col],
                                 mode="lines", name=col))
        
    
    fig1.update_layout( height=800, width =1000, autosize=False,
        title="Momentum Rank", xaxis_title='Date', yaxis_title="Rank(1=Best)", yaxis_autorange="reversed",
        template ="plotly_white", hovermode="x unified")
    
    fig1.add_hrect(y0=0.5, y1=top_n+0.5, fillcolor="green", opacity=0.15,
                   layer="below", line_width=0)
    
    fig1.show()



    #Market Regime plot
    
    prices_market = prices[market]
    #market_signal_daily = (market_signal.reindex(prices_market.index).ffill().fillna(False))
    
    fig = go.Figure()

    fig.add_trace(
    go.Scatter(
        x=prices_market.index,
        y=prices_market,
        mode="lines",
        name="Market ", line=dict(width=2)
        ))
    
    #daily signal
    """fig.add_trace(
    go.Scatter(
        x=prices_market.loc[market_signal_daily].index,
        y=prices_market.loc[market_signal_daily],
        mode="lines",
        name="Market ", line=dict(width=2)
        ))  """      

    #for month-end plot           

    fig.add_trace(
    go.Scatter(
        x=market_signal[market_signal].index,
        y=prices_market.resample("ME").last().loc[market_signal],
        mode="markers",
        name="Risk ON (Monthly)",
        marker=dict(size=6)
            )
                    )

    fig.update_layout(
    title="Market Regime Filter",
    xaxis_title="Date",
    yaxis_title="Market Price",
    template="plotly_white"
                )

    fig.show()
    

    return equity_curve

In [97]:
stocks = ['LLY', 'JNJ', 'AMGN','MRK','NVO','PFE']
market ='SPY'
ticker = stocks + [market]

In [98]:
prices = stock_data(ticker, "2010-01-01")
prices.tail(10)

[*********************100%***********************]  7 of 7 completed


Ticker,AMGN,JNJ,LLY,MRK,NVO,PFE,SPY
Date,,,,,,,
2025-12-15,325.309998,214.169998,1062.189941,100.260002,50.369999,26.430000,678.724426
2025-12-16,326.739990,209.300003,1054.290039,98.269997,48.959999,25.530001,676.869934
2025-12-17,326.010010,210.330002,1041.790039,99.180000,47.770000,25.040001,669.421936
2025-12-18,324.420013,208.309998,1056.880005,100.690002,47.610001,25.040001,674.476929
2025-12-19,327.380005,206.369995,1071.439941,101.089996,48.090000,25.190001,680.590027
2025-12-22,331.390015,207.320007,1076.479980,104.720001,48.099998,25.209999,684.830017
2025-12-23,331.489990,205.779999,1071.640015,105.040001,51.610001,24.879999,687.960022
2025-12-24,333.959991,207.779999,1076.979980,106.449997,52.560001,25.030001,690.380005
2025-12-26,332.929993,207.630005,1077.750000,106.779999,52.400002,25.090000,690.309998


In [109]:
equity_curve = dual_momemtum(prices,stocks,market,mom_lookback=6, ma_window=10, top_n=4)
equity_curve

Date
2010-01-04    1.000000
2010-01-05    1.000000
2010-01-06    1.000000
2010-01-07    1.000000
2010-01-08    1.000000
                ...   
2025-12-22    7.454315
2025-12-23    7.422334
2025-12-24    7.490291
2025-12-26    7.493457
2025-12-29    7.464730
Length: 4022, dtype: float64

In [73]:
def plot_return(equity_curve):
    """Plot the equity curve and drawdown"""
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=equity_curve.index, y = equity_curve, mode="lines", name='Dual Momentum'))

    fig.update_layout(title = "Dual Momentum Equity curve",
                      xaxis_title="Date", yaxis_title ='Portfolio value', yaxis_type = "log",
                      template="plotly_white", hovermode ='x unified')
    fig.show()

    rolling_max = equity_curve.cummax()
    drawdown = (equity_curve / rolling_max )-1

    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(x = drawdown.index, y = drawdown, fill ='tozeroy', name = 'Drawdown'))
    fig1.update_layout(title='Drawdown', 
                       xaxis_title = 'Date', yaxis_title = 'Drawdown', template='plotly_white',
                       hovermode ='x unified')
    fig1.show()




In [ ]:
#Healthcare Stocks
stocks = ['LLY', 'JNJ', 'AMGN','MRK','NVO','PFE']
market ='SPY'
ticker = stocks + [market]

Tech Stocks

In [110]:
stocks = ['AAPL', 'MSFT', "NVDA", 'TSLA', 'GOOG', "AMZN", "META"]
market ='SPY'
ticker = stocks + [market]


In [111]:
prices = stock_data(ticker, "2000-01-01")
prices.tail(10)

[*********************100%***********************]  8 of 8 completed


Ticker,AAPL,AMZN,GOOG,META,MSFT,NVDA,SPY,TSLA
Date,,,,,,,,
2025-12-15,274.109985,222.539993,309.320007,647.510010,474.820007,176.289993,678.724426,475.309998
2025-12-16,274.609985,222.559998,307.730011,657.150024,476.390015,177.720001,676.869934,489.880005
2025-12-17,271.839996,221.270004,298.059998,649.500000,476.119995,170.940002,669.421936,467.260010
2025-12-18,272.190002,226.759995,303.750000,664.450012,483.980011,174.139999,674.476929,483.369995
2025-12-19,273.670013,227.350006,308.609985,658.770020,485.920013,180.990005,680.590027,481.200012
2025-12-22,270.970001,228.429993,311.329987,661.500000,484.920013,183.690002,684.830017,488.730011
2025-12-23,272.359985,232.139999,315.679993,664.940002,486.850006,189.210007,687.960022,485.559998
2025-12-24,273.809998,232.380005,315.670013,667.549988,488.019989,188.610001,690.380005,485.399994
2025-12-26,273.399994,232.520004,314.959991,663.289978,487.709991,190.529999,690.309998,475.190002


In [112]:
equity_curve = dual_momemtum(prices, stocks, market, mom_lookback=6, ma_window=10, top_n= 3)

In [113]:
plot_return(equity_curve)